In [ ]:
# "status": "ok",\n  "seasons": {\n    "winter": "0 12 2,15 1,2,12 *",\n    "autumn": "0 12 */7 3-5 *",\n    "summer": "0 12 */2 6-8 *",\n    "spring": "0 12 */7 9-11 *"
# need to specify by user when he or she last water the plant because I need datetime from this period

/tmp/ipykernel_177178/2213911182.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_date = datetime.utcnow()


datetime.datetime(2025, 2, 18, 15, 40, 6, 52903)

In [2]:
%run "template.ipynb"

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a translator of home flower from English to Russian"
            "You are translate an flower from the text."
            "Send only the name of the flower or null"
            "Flowers should be in the Russia language",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [4]:
from sqlmodel import SQLModel, Field
from uuid import UUID, uuid4
from datetime import date


class Plant(SQLModel, table=True):
    __tablename__ = "plants"
    __table_args__ = {"extend_existing": True}
    id: UUID = Field(primary_key=True, default_factory=uuid4)
    name: str | None
    origin_name: str
    bibliography: str
    slug: str
    year: date
    image_url: str
    family: str
    genus: str
    rank: str

In [5]:
from sqlmodel import select

async with session_maker() as session:
    plants = await session.exec(select(Plant))

In [6]:
plants = plants.all()

In [7]:
len(plants)

100

In [8]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="qwen2.5",
    temperature=0.05,
)

In [13]:
from langchain_core.messages.ai import AIMessage
from tqdm import tqdm

df = pd.DataFrame(columns=["old_name", "new_name"])
for plant in tqdm(plants, total=len(plants), desc="Anchoring", unit="page"):
    text = f"Переведи на русский это домашнее растение - {plant.origin_name}"
    prompt = prompt_template.invoke({"text": text})
    response = llm.invoke(prompt)
    df.loc[plant.id] = (plant.origin_name, response.content)

Anchoring: 100%|██████████| 100/100 [03:29<00:00,  2.10s/page]


In [15]:
df

,old_name,new_name
5d58b28f-9b31-4b4a-975f-d2c065a39330,Meadow brome,Костяшка Meadows
9ec5a11c-5a42-4af6-9af5-47beb0c7f169,Easter-bell,Весенняя колокольчик
aa126c55-99ed-429f-b1ea-7204f01a21ed,Field speedwell,Полынь полевая
eeb6a4f7-fbd3-44a3-807b-56a5bde966b4,Umbrella pine,Купальníк
1da23ba2-886e-4cf8-b023-30e6ae63fe2e,White cockle,Белая коклюшка
...,...,...
a62c8b52-8ac4-421c-8124-a3cfcf5a7dbb,Bitter dock,Лапчатка горькая
674f956c-fee9-43cd-9f7a-a7a850b4e45f,Oxeye daisy,Льнянка
cf76c942-6de2-40fe-83f3-a117143f272a,Male fern,Мужской ферн
480990f0-9541-4fb9-a643-3b32b5fc2231,Felonherb,null


In [16]:
import re

regex = re.compile("^[а-яА-ЯёЁ ]+$")
for row in df.iterrows():
    new_name = row[1]["new_name"]
    if new_name is not None and (
        new_name in ("null", "Нулл", "Ничего")
        or len(new_name.split()) > 5
        or regex.match(new_name) == None
    ):
        row[1]["new_name"] = None


In [17]:
df

,old_name,new_name
5d58b28f-9b31-4b4a-975f-d2c065a39330,Meadow brome,None
9ec5a11c-5a42-4af6-9af5-47beb0c7f169,Easter-bell,Весенняя колокольчик
aa126c55-99ed-429f-b1ea-7204f01a21ed,Field speedwell,Полынь полевая
eeb6a4f7-fbd3-44a3-807b-56a5bde966b4,Umbrella pine,None
1da23ba2-886e-4cf8-b023-30e6ae63fe2e,White cockle,Белая коклюшка
...,...,...
a62c8b52-8ac4-421c-8124-a3cfcf5a7dbb,Bitter dock,Лапчатка горькая
674f956c-fee9-43cd-9f7a-a7a850b4e45f,Oxeye daisy,Льнянка
cf76c942-6de2-40fe-83f3-a117143f272a,Male fern,Мужской ферн
480990f0-9541-4fb9-a643-3b32b5fc2231,Felonherb,None


In [18]:
df.to_csv("flowers.csv", index=True)

In [ ]:
from yandex_cloud_ml_sdk import YCloudML

PROMPT_EXAMPLE = """
Как часто поливать кактусы
Единого мнения не существует. Это зависит от того, в каком состоянии находится растение, какая температура и влажность воздуха, как много света оно получает. 

Зима. В основном зимой кактус спит. А потому полив полностью прекращают с ноября. В то же время, если кактус зимует в теплом помещении и к тому же цветет, полив стоит продолжать, но делать это реже: достаточно одной процедуры раз в 2-3 недели.

Весна. Понемногу давать растению воду начинают с весны, когда кактус просыпается. О том, что этот момент наступил, можно понять по зеленеющей макушке растения. Кроме того, на его верхушке появляются молодые колючки. Частота полива – раз в 12-14 дней.

Лето. Максимальная частота полива, процедура нужна еженедельно. Если стоит слишком жаркая погода, увлажнять землю в горшке стоит каждые 5-6 дней.

Осень. Регулярность постепенно сокращают перед зимовкой растения. Это особенно важно, если кактус будет находиться в прохладном помещении – при слишком частом осеннем поливе его ствол может деформироваться. 
"""

prompt_text = """
Ты создаешь график полива на разные времена года, это Зима, Весна, Лето и Осень cron синтаксисом растений которые у тебя спрашивает пользователь. Возращай данные в указанном JSON с указанием точных дат
Пример:

---

Какой график полива у растения: Кактус?

Ответ:
{
  "status": "ok",
  "seasons": {
    "winter": "0 12 2,15 1,2,12 *",
    "autumn": "0 12 */7 3-5 *",
    "summer": "0 12 */2 6-8 *",
    "spring": "0 12 */7 9-11 *"
   }
}

---
"""


def change_in_place_response(seasons: dict):
    cron = seasons["winter"].split(" ")
    cron[-2] = "1,2,12"
    seasons["winter"] = " ".join(cron)

    cron = seasons["autumn"].split(" ")
    cron[-2] = "3-5"
    seasons["autumn"] = " ".join(cron)

    cron = seasons["summer"].split(" ")
    cron[-2] = "6-8"
    seasons["summer"] = " ".join(cron)

    cron = seasons["spring"].split(" ")
    cron[-2] = "9-11"
    seasons["spring"] = " ".join(cron)


def main():
    sdk = YCloudML(
        folder_id="b1g4o9t4vgcbnkpunqeu",
        auth="y0_AgAAAAB23CtmAATuwQAAAAEUyAepAAAhD4Z1VbBNGLF5BQT5m8ACGp__wA",
    )

    result = (
        sdk.models.completions("yandexgpt")
        .configure(temperature=0.5)
        .run(
            [
                {
                    "role": "system",
                    "text": prompt_text,
                },
                {
                    "role": "user",
                    "text": "Какой график полива у растения: Алоэ Вера",
                },
            ]
        )
    )
    # if not result['ok']:
    #     return

    for alternative in result:
        print(alternative)


if __name__ == "__main__":
    main()


/home/bevzd/workspace/pythonProjects/service-auth/services/migrations/src/seed/calendar/.venv/lib/python3.12/site-packages/yandex_cloud_ml_sdk/_auth.py:181: UserWarning: Sharing your personal OAuth token is not safe,
and gives anyone access to your cloud infrastructure and data.

Use YandexCloudCLIAuth for personal authentication,
MetadataAuth when running your code inside Yandex Cloud infrastructure or
APIKeyAuth for external-hosted automations.

Please, follow our guide if your OAuth-token is leaked
(https://yandex.cloud/en/docs/iam/operations/compromised-credentials)

  warnings.warn(


Alternative(role='assistant', text='```\n{\n  "status": "ok",\n  "seasons": {\n    "winter": "0 12 2,15 1,2,12 *",\n    "autumn": "0 12 */7 3-5 *",\n    "summer": "0 12 */2 6-8 *",\n    "spring": "0 12 */7 9-11 *"\n   }\n}\n```', status=<AlternativeStatus.FINAL: 3>)
